In [4]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import urllib.request
from urllib.request import urlopen
import pyautogui
from requests.exceptions import ConnectionError
# from pywebcopy import save_webpage

In [5]:

internal_urls = set()
external_urls = set()
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

#The above function works as follows. We check if the scheme is present and there is a value in the network location part
# url = "https://umd.edu/virusinfo"
# urlparse(url)
# ParseResult(scheme='https', netloc='umd.edu', path='/virusinfo', 
# params='', query='', fragment='')

#The function gives all urls
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        print(f"[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls

total_urls_visited = 0
def crawl(url, max_urls=50):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    try:
        links = get_all_urls(url)
        for link in links:
            if total_urls_visited > max_urls:
                break
            crawl(link, max_urls=max_urls)
    except:
        print("caught")


In [6]:
base_urls = ["https://www.cameron.edu/covid-19","https://www.canisius.edu/student-experience/student-support-services/student-health","https://www.carrollu.edu/student-services/health/coronavirus"]


In [7]:
for base_url in base_urls:
    internal_urls.add(base_url)
    parsedurl = urlparse(base_url)
    print(parsedurl)
    base_url_main = parsedurl.scheme+"://"+parsedurl.netloc
    print(base_url_main)
    # base_url_text = base_url.split("//",1)[1]
    base_url_text = parsedurl.netloc+parsedurl.path
    print(base_url_text)
    # base_url_text_domain = base_url_text.split("/",1)[0]
    base_url_text_domain = parsedurl.netloc
    print(base_url_text_domain)
    crawl(base_url)
    print("[+] Total External links:", len(external_urls))
    print("[+] Total Internal links:", len(internal_urls))
    print("[+] Total:", len(external_urls) + len(internal_urls))
    count = 0
    last_count = 989874
    immediate_urls = set() #Linked Associated with the current page, https://umd.edu/virusinfo/ in this case

    for i in range(0,4):
        for url in internal_urls:
            if base_url_text in url:
                immediate_urls.add(url)
        count = len(immediate_urls)
        print(count,i)
        if(count == last_count):
            break
        last_count = count
        for immediate_url in immediate_urls:
            try:
                crawl(immediate_url)
                print("[+] NEW Total External links:", len(external_urls))
                print("[+] NEW Total Internal links:", len(internal_urls))
                print("[+] NEW Total:", len(external_urls) + len(internal_urls))
                if len(internal_urls)>1000:
                    break
            except requests.exceptions.ConnectionError:
                continue

    def removeAfterN(yourStr, nth, occurenceOf):
        return occurenceOf.join(yourStr.split(occurenceOf)[:nth])
    
    
    semifinal_immediate_urls = set()

    for immediate_url in immediate_urls:
        stripped_url = removeAfterN(immediate_url,2,"http")


        semifinal_immediate_urls.add(stripped_url)
    #     final_immediate_urls.add(rest)
    #     print(rest)



    for i in semifinal_immediate_urls:
        print(i)
    total_count = len(semifinal_immediate_urls)
    print(total_count)
    
    
    final_immediate_urls = set()
    for semifinal_immediate_url in semifinal_immediate_urls:
        if "tel:" not in semifinal_immediate_url:
            final_immediate_urls.add(semifinal_immediate_url)


    for i in final_immediate_urls:
        print(i)
    total_count = len(final_immediate_urls)
    print(total_count)       
    
    
    
    parent_folder = "Masters" #Change based on folder
    name_of_folder = f'{base_url_text_domain}_Screenshots'
    options = webdriver.ChromeOptions()
    options.headless = True
    driver = webdriver.Chrome(options=options,executable_path='chromedriver.exe') # #Local Path of chrome driver
    if not os.path.exists(f'{parent_folder}\\{name_of_folder}'):
        os.makedirs(f'{parent_folder}\\{name_of_folder}')
    
    
    
    webpage_save = set()
    webpage_save_count = 0
    webpage_cannot_save = 0
    for i in final_immediate_urls:
        if ' ' in i:
            continue
        try:
            code=urlopen(i).getcode()
            webpage_save.add(i)
            webpage_save_count = webpage_save_count+1
            print(f"{code} can_save - {i}")
        except (urllib.error.HTTPError,urllib.error.URLError) as e:
            print("discarded")
            webpage_cannot_save = webpage_cannot_save+1
            continue

    print(f'Webpages save as HTML :{webpage_save_count}')
    print(f'Webpages unable to save as HTML :{webpage_cannot_save}')

    
    pdf_count = 0
    pdf_urls = set()
    for webpage in webpage_save:
        try:
            if webpage[-4:] == ".pdf":
                url = webpage
                file_name = url.replace(base_url,'')
                file_name = file_name.replace('/','')
                if file_name == '':
                    file_name = "Home"
                pdf_urls.add(webpage)
                print(webpage)
                r = requests.get(url, stream=True)
                with open(f'{parent_folder}\\{name_of_folder}\\{file_name}.pdf', 'wb') as fd:
                    for chunk in r.iter_content(2000):
                        fd.write(chunk)
                pdf_count = pdf_count + 1
        except:
            print("caught")

    print(f"Pdfs saved: {pdf_count}")
    
    
    
    final_immediate_urls = final_immediate_urls - pdf_urls
    webpage_save = webpage_save - pdf_urls
    web_save_count = len(webpage_save)
    total_count = len(final_immediate_urls)
    print(total_count) 
    
    
    
    final_webpage_save_count = webpage_save_count - pdf_count
    print(final_webpage_save_count)
    
    
    webpage_save_counter_loop = 0
    for webpage in webpage_save:
        try:
            url = webpage
            file_name = url.replace(base_url,'')
            file_name = file_name.replace('/','')
            if file_name == '':
                    file_name = "Home"
            urllib.request.urlretrieve (webpage, f"{parent_folder}\\{name_of_folder}\\{file_name}.html")
            webpage_save_counter_loop = webpage_save_counter_loop+1
            print(f'{webpage_save_counter_loop} of {final_webpage_save_count} - {webpage}')
        except:
            print("caught")

    print("Saving Completed")
    
    
    count = 0
    for i in final_immediate_urls:
        try:
            url = i
            file_name = url.replace(base_url,'')
            file_name = file_name.replace('/','')
            if file_name == '':
                    file_name = "Home"
            count = count + 1
            print(f"{count} of {total_count}")
            print(f'Visiting {base_url_text}/{file_name}')
            print(f'...Taking a screenshot')
            driver.get(url)
            S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
            driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
            driver.find_element_by_tag_name('body').screenshot(f'{parent_folder}\\{name_of_folder}\\{file_name}.png')
            print(f'Screenshot of {file_name} page taken! \n')
        except:
            print("caught")
    #     print("Saving HTML")
    #     pyautogui.hotkey('ctrl', 's')
    #     time.sleep(1)
    #     pyautogui.typewrite(f'{file_name}.html')
    #     pyautogui.hotkey('enter')
    driver.quit()

    print(f'Task Completed! Files stored in the {parent_folder}\\{name_of_folder} Folder')

ParseResult(scheme='https', netloc='www.cameron.edu', path='/covid-19', params='', query='', fragment='')
https://www.cameron.edu
www.cameron.edu/covid-19
www.cameron.edu
[INT] Internal link: https://www.cameron.edu/
[INT] Internal link: https://www.cameron.edu/campus-map
[INT] Internal link: https://www.cameron.edu/duncan
[INT] Internal link: https://www.cameron.edu/email
[INT] Internal link: https://www.cameron.edu/its
[INT] Internal link: https://www.cameron.edu/university-calendar
[INT] Internal link: https://www.cameron.edu/academics
[INT] Internal link: https://www.cameron.edu/academics/calendar
[INT] Internal link: https://www.cameron.edu/academics/degrees-majors
[INT] Internal link: https://www.cameron.edu/academics/minors
[INT] Internal link: https://www.cameron.edu/academics/online-itv-distance-learning
[INT] Internal link: https://www.cameron.edu/academics/school-departments
[INT] Internal link: https://www.cameron.edu/academics/university-catalogs
[INT] Internal link: https

[INT] Internal link: https://www.cameron.edu/facility-reservation
[INT] Internal link: https://www.cameron.edu/facility-reservation/mcmahon-centennial-complex
[INT] Internal link: https://www.cameron.edu/facility-reservation/cetes-conference-center
[INT] Internal link: https://www.cameron.edu/facility-reservation/shepler-ballroom
[INT] Internal link: https://www.cameron.edu/facility-reservation/catering-information
[INT] Internal link: https://www.cameron.edu/facility-reservation/room-information-and-pricing
[INT] Internal link: https://www.cameron.edu/facility-reservation/staff
[INT] Internal link: https://www.cameron.edu/facility-reservation/mcmahon-centennial-complex/mccasland-foundation-ballroom
[INT] Internal link: https://www.cameron.edu/facility-reservation/mcmahon-centennial-complex/buddy-green-room
[INT] Internal link: https://www.cameron.edu/facility-reservation/mcmahon-centennial-complex/ibcrma-executive-board-room
[INT] Internal link: https://www.cameron.edu/facility-reserv

[INT] Internal link: http://www.cameron.edu/oeo/affirmative_action
[INT] Internal link: http://www.cameron.edu/oeo/message-from-the-president
[INT] Internal link: http://www.cameron.edu/academic_affairs/articulation
[INT] Internal link: http://www.cameron.edu/academic_affairs/curriculum
[INT] Internal link: http://www.cameron.edu/academic_affairs/service-learning
[INT] Internal link: http://www.cameron.edu/academic_affairs/faculty-handbook
[INT] Internal link: http://www.cameron.edu/academic_affairs/councils-and-committees
[INT] Internal link: http://www.cameron.edu/academic_affairs/oklahoma-research-day
[INT] Internal link: http://www.cameron.edu/academic_affairs/internal-grants
[INT] Internal link: http://www.cameron.edu/academic_affairs/research
[INT] Internal link: http://www.cameron.edu/graduate-and-professional-studies
[INT] Internal link: http://www.cameron.edu/school-of-arts-and-sciences
[INT] Internal link: http://www.cameron.edu/office-of-teaching-and-learning
[INT] Internal 

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[INT] Internal link: https://www.cameron.edu/users/sign_in
[INT] Internal link: https://www.cameron.edu/opportunities
[INT] Internal link: https://www.cameron.edu/opportunities/flexible
[INT] Internal link: https://www.cameron.edu/opportunities/external
[INT] Internal link: https://www.cameron.edu/opportunities/1222
[INT] Internal link: https://www.cameron.edu/opportunities/625
[INT] Internal link: https://www.cameron.edu/opportunities/1634
[INT] Internal link: https://www.cameron.edu/opportunities/1637
[INT] Internal link: https://www.cameron.edu/opportunities/1588
[INT] Internal link: https://www.cameron.edu/opportunities/1564
[INT] Internal link: https://www.cameron.edu/opportunities/1600
[INT] Internal link: https://www.cameron.edu/opportunities/1360
[INT] Internal link: https://www.cameron.edu/opportunities/1603
[INT] Internal link: https://www.cameron.edu/opportunities/1273
[INT] Internal link: https://www.cameron.edu/opportunities/1474
[INT] Internal link: https://www.cameron.ed

...

200 can_save - http://www.canisius.edu/student-experience/student-support-services/student-health/faqs
200 can_save - http://www.canisius.edu/student-experience/student-support-services/student-health/health-insurance
200 can_save - http://www.canisius.edu/student-experience/student-support-services/student-health/preparing-for-coronavirus
200 can_save - https://www.canisius.edu/student-experience/student-support-services/student-health
200 can_save - https://www.canisius.edu/student-experience/student-support-services/student-health/health-requirements
200 can_save - http://www.canisius.edu/student-experience/student-support-services/student-health/coronavirus-information-canisius
200 can_save - http://www.canisius.edu/student-experience/student-support-services/student-health/coronavirus-abroad-information
200 can_save - http://www.canisius.edu/student-experience/student-support-services/student-health/medical-emergencies-after-hours-care
200 can_save - https://www.canisius.edu/stude

Screenshot of http:www.canisius.edustudent-experiencestudent-support-servicesstudent-healthhealth-insurance page taken! 

3 of 32
Visiting www.canisius.edu/student-experience/student-support-services/student-health/http:www.canisius.edustudent-experiencestudent-support-servicesstudent-healthpreparing-for-coronavirus
...Taking a screenshot
Screenshot of http:www.canisius.edustudent-experiencestudent-support-servicesstudent-healthpreparing-for-coronavirus page taken! 

4 of 32
Visiting www.canisius.edu/student-experience/student-support-services/student-health/Home
...Taking a screenshot
Screenshot of Home page taken! 

5 of 32
Visiting www.canisius.edu/student-experience/student-support-services/student-health/health-requirements
...Taking a screenshot
Screenshot of health-requirements page taken! 

6 of 32
Visiting www.canisius.edu/student-experience/student-support-services/student-health/http:www.canisius.edustudent-experiencestudent-support-servicesstudent-healthcoronavirus-informat

[INT] Internal link: https://www.carrollu.edu/
[INT] Internal link: https://www.carrollu.edu/admissions
[INT] Internal link: https://www.carrollu.edu/admissions/freshmen
[INT] Internal link: https://www.carrollu.edu/admissions/graduate
[INT] Internal link: https://www.carrollu.edu/admissions/transfer
[INT] Internal link: https://www.carrollu.edu/admissions/part-time
[INT] Internal link: https://www.carrollu.edu/admissions/international
[INT] Internal link: https://www.carrollu.edu/admissions/special
[INT] Internal link: https://www.carrollu.edu/admissions/pre-college
[INT] Internal link: https://www.carrollu.edu/financial-aid
[INT] Internal link: https://www.carrollu.edu/financial-aid/net-price-calculator
[INT] Internal link: https://www.carrollu.edu/academics
[INT] Internal link: https://www.carrollu.edu/academics/undergraduate-programs
[INT] Internal link: https://www.carrollu.edu/academics/graduate-programs
[INT] Internal link: https://www.carrollu.edu/academics/arts-sciences
[INT] 

[INT] Internal link: https://www.carrollu.edu/student-services/health/coronavirus/tel:2625471211
[+] NEW Total External links: 326
[+] NEW Total Internal links: 665
[+] NEW Total: 991
[+] NEW Total External links: 331
[+] NEW Total Internal links: 665
[+] NEW Total: 996
[INT] Internal link: https://www.carrollu.edu/student-services/health/coronavirus/town-hall-fall-2020-students/tel:2625471211
[+] NEW Total External links: 334
[+] NEW Total Internal links: 666
[+] NEW Total: 1000
[INT] Internal link: https://www.carrollu.edu/student-services/health/coronavirus/student-cares-grants/tel:2625471211
[+] NEW Total External links: 334
[+] NEW Total Internal links: 667
[+] NEW Total: 1001
[+] NEW Total External links: 334
[+] NEW Total Internal links: 667
[+] NEW Total: 1001
[INT] Internal link: https://www.carrollu.edu/student-services/health/coronavirus/refund-information/tel:2625471211
[+] NEW Total External links: 336
[+] NEW Total Internal links: 668
[+] NEW Total: 1004
[+] NEW Total Ext

200 can_save - https://www.carrollu.edu/student-services/health/coronavirus/study-abroad-and-cce
200 can_save - https://www.carrollu.edu/student-services/health/coronavirus/town-hall-fall-2020-students/faq
200 can_save - https://www.carrollu.edu/student-services/health/coronavirus/student-cares-grants/cares-act-reporting
200 can_save - https://www.carrollu.edu/student-services/health/coronavirus/recsports-faq
200 can_save - https://www.carrollu.edu/student-services/health/coronavirus/refund-information/faq
200 can_save - https://www.carrollu.edu/student-services/health/coronavirus/student-org-faq
200 can_save - https://www.carrollu.edu/student-services/health/coronavirus/2020-03-06-study-abroad-participant
200 can_save - https://www.carrollu.edu/student-services/health/coronavirus/2020-03-04-spring-break-travel
200 can_save - https://www.carrollu.edu/student-services/health/coronavirus/transition-to-all-online-learning
200 can_save - https://www.carrollu.edu/student-services/health/cor